In [2]:
import torch
import os
from datasets import load_dataset

# Create baseline/ordered tensor of scores
def load_indexed_tensor(task_name):
    path = "results/" + task_name +"/"
    
    # if "results/" + task_name + "/baseline_scores.pt" exists, load
    if os.path.exists(path+"baseline_scores.pt"):
        print("Loading existing tensor")
        baseline_scores = torch.load(path+"baseline_scores.pt")
        return baseline_scores
    else:
        print("Creating new tensor")
        sorted_indices = torch.load(path+"sorted_indices.pt")
        sorted_scores = torch.load(path+"sorted_scores.pt")
        baseline_scores_tensor = torch.zeros_like(sorted_scores)
        baseline_scores_tensor.scatter_(1, sorted_indices, sorted_scores)
        torch.save(baseline_scores_tensor, path+"baseline_scores.pt")
    return baseline_scores_tensor # in the format of the pairwise plots 

# Find index of train story with most variance over the test blimp influence scores (ie columns of pairwise plots with most color variation)
def variance_stories(n, baseline_scores):
    column_std = baseline_scores.std(dim=0)
    _, top_std_idxs = torch.topk(column_std, n)
    # Identify the stories
    split = "train"
    tinystories = load_dataset("s-ostrove/moreStories")[split]
    top_stories = [tinystories[i]['text'] for i in top_std_idxs.tolist()]   
    return top_stories

def top_variance_blimp(n, baseline_scores, task):
    row_std = baseline_scores.std(dim=1)
    _, top_std_idxs = torch.topk(row_std, n)
    # Identify the blimp pairs
    blimp = load_dataset("blimp", task)
    top_pairs = [blimp['train'][i] for i in top_std_idxs.tolist()]   
    # just take 'sentence_good' and 'sentence_bad'
    top_pairs = [{"sentence_good": pair['sentence_good'], "sentence_bad": pair['sentence_bad'], "id": pair['pair_id']} for pair in top_pairs]
    return top_pairs

In [2]:
anaphor_gender_baseline_scores = load_indexed_tensor("anaphor_gender_agreement")
#variance_stories(10, anaphor_gender_baseline_scores)

Loading existing tensor


/tmp/ipykernel_3065793/2002522197.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  baseline_scores = torch.load(path+"baseline_scores.pt")


*Look into whether this might be due to 'hard' words, use of itself, contraction, or names 

In [5]:
# Examine top and bottom stories contributing to each id
def examine_variance(n_blimp, n_stories, baseline_scores, task):
    top_pairs = top_variance_blimp(n_blimp, baseline_scores, task)
    ids = [pair['id'] for pair in top_pairs]
    tinystories = load_dataset("s-ostrove/moreStories")['train']
    # list of rows in baseline_scores
    blimp_rows = [baseline_scores[i] for i in ids]
    [max_scores, max_ids] = [torch.topk(blimp_row, n_stories) for blimp_row in blimp_rows]
    top_stories = [[tinystories[i] for i in max_ids.tolist()] for max_ids in max_ids]
    return top_stories

In [6]:
def examine_variance(n_blimp, n_stories, baseline_scores, task):
    top_pairs = top_variance_blimp(n_blimp, baseline_scores, task)
    ids = [pair['id'] for pair in top_pairs]
    tinystories = load_dataset("s-ostrove/moreStories")['train']
    
    # Extract rows corresponding to selected ids
    blimp_rows = [baseline_scores[i] for i in ids]

    # For each row, get top n_stories scores and their indices
    top_stories = []
    bottom_stories = []
    for blimp_row in blimp_rows:
        values, indices = torch.topk(blimp_row, n_stories)
        stories = [tinystories[i]['text'] for i in indices.tolist()]
        top_stories.append(stories)
        # bottom n_stories
        values, indices = torch.topk(blimp_row, n_stories, largest=False)
        stories = [tinystories[i]['text'] for i in indices.tolist()]
        bottom_stories.append(stories)
    
    return top_stories, bottom_stories


In [46]:
top_variance_blimp(5, anaphor_gender_baseline_scores, "anaphor_gender_agreement")

[{'sentence_good': "Kristen isn't aggravating herself.",
  'sentence_bad': "Kristen isn't aggravating itself.",
  'id': 107},
 {'sentence_good': 'Kayla insults herself.',
  'sentence_bad': 'Kayla insults itself.',
  'id': 483},
 {'sentence_good': 'Kenneth insulted himself.',
  'sentence_bad': 'Kenneth insulted itself.',
  'id': 492},
 {'sentence_good': "Carlos isn't irritating himself.",
  'sentence_bad': "Carlos isn't irritating itself.",
  'id': 198},
 {'sentence_good': 'Kirsten dislikes herself.',
  'sentence_bad': 'Kirsten dislikes itself.',
  'id': 193}]

In [51]:
top, bottom = examine_variance(n_blimp=5, n_stories=10, baseline_scores=anaphor_gender_baseline_scores, task="anaphor_gender_agreement")
print("TOP INFLUENTIAL STORIES\n")
top

TOP INFLUENTIAL STORIES



[['',
  '',
  '',
  '',
  '',
  "Nicky was walking outside and saw a puppy playing in the park. Nicky saw how friendly the puppy was and wanted to be friends. Nicky waved at the puppy and the puppy came running over to him.\n\nNicky: Hi puppy!\n\nPuppy: (Bark, bark!)\n\nNicky laughed and was so happy to make a new friend.\n\nNicky: Do you want to play?\n\nPuppy: (Bark, bark!)\n\nNicky and the puppy played together for a long time. Nicky and the puppy ran around and chased each other until they were both out of breath. Nicky was so happy and his friends were envious of him.\n\nNicky: I've gained a new best friend!\n\nPuppy: (Bark, bark!)",
  'Bert and Bean were in the kitchen. Bean wanted to decorate a pear for a special snack. Bert wanted to help her, so he carefully drew some stars and stripes on the pear with a pen. After that, Bean was very happy and thanked Bert for being so helpful. Next, Bean put the pear on a plate. On the plate, Bean put some delicious yogurt and sprinkles.\n\n

In [52]:
print("BOTTOM INFLUENTIAL STORIES\n")
bottom
# includes themsel(f)(ves)

BOTTOM INFLUENTIAL STORIES



[['Once there was a little girl called Letsty. When Letsty was very confused, she always looked for her safe. But one day when Letsty couldn\'t find her safe, she asked her mum, "Where\'s my safe?". \n\nMum said, "It\'s okay, Letsty. I had to remove your safe to clean the house." \n\nLetsty was wondering what it meant to remove, so she asked her mum again. Her mum replied, "Remove means to take away something very carefully." \n\nLetsty understood what to do when she was confused. And she felt safe knowing that her mum would always look after her and remove her worries.\n\nThe moral of the story is that no matter how confused we are, we can always rely on our family to help us through it.',
  'Mom and Dad were not happy. Something was wrong with the cabinet in their room. It was broken and they had to find a way to fix it. Mom and Dad removed all of their things from the cabinet. They put the broken pieces in a box. Then they tried to fix the cabinet.\n\nMom and Dad used glue and put t

### ANAPHOR_NUMBER_AGREEMENT

In [56]:
task = "anaphor_number_agreement"
anaphor_number_baseline_scores = load_indexed_tensor(task)
top_variance_blimp(5, anaphor_number_baseline_scores, task)

Loading existing tensor


/tmp/ipykernel_3065793/965520794.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  baseline_scores = torch.load(path+"baseline_scores.pt")


[{'sentence_good': 'The Impressionists concealed themselves.',
  'sentence_bad': 'The Impressionists concealed itself.',
  'id': 178},
 {'sentence_good': 'Harvard University shocks itself.',
  'sentence_bad': 'Harvard University shocks themselves.',
  'id': 258},
 {'sentence_good': "These companies wouldn't aggravate themselves.",
  'sentence_bad': "These companies wouldn't aggravate itself.",
  'id': 123},
 {'sentence_good': 'Roger insulted himself.',
  'sentence_bad': 'Roger insulted themselves.',
  'id': 489},
 {'sentence_good': 'Some government worries itself.',
  'sentence_bad': 'Some government worries themselves.',
  'id': 481}]

In [57]:
top, bottom = examine_variance(n_blimp=5, n_stories=10, baseline_scores=anaphor_number_baseline_scores, task=task)
print("TOP INFLUENTIAL STORIES\n")
top

TOP INFLUENTIAL STORIES



[['Bert and Bean were in the kitchen. Bean wanted to decorate a pear for a special snack. Bert wanted to help her, so he carefully drew some stars and stripes on the pear with a pen. After that, Bean was very happy and thanked Bert for being so helpful. Next, Bean put the pear on a plate. On the plate, Bean put some delicious yogurt and sprinkles.\n\nThen the two friends enjoyed their snack together. They talked and laughed while they ate. The pear was the best part of the snack and they both agreed it looked cool with its decorations. After that, they thanked each other again for being so helpful.',
  "Once there was a brave young man called Alfred. He was very enthusiastic about protecting his village from danger. One day, the sky above the village turned black and ashes started to fall from the sky. Alfred was alarmed and quickly moved to protect the people of his village. He found a big bucket and he eagerly ran around, catching the ashes before they hit the ground. Everyone was am

In [58]:
print("BOTTOM INFLUENTIAL STORIES\n")
bottom

BOTTOM INFLUENTIAL STORIES



[['Once upon a time, there was a little girl named Sue. Sue loved to read. She went to the library every day to find new books. The library was a big house with many books.\n\nOne day, Sue found a very special book. It was delicate, with pretty pictures inside. Sue wanted to take the book home, but she didn\'t want it to get hurt. So, she decided to wrap the book in her soft scarf.\n\nAs Sue walked home, she met her friend Tom. Tom asked, "What do you have there, Sue?" Sue smiled and said, "I found a delicate book at the library. I wrapped it in my scarf to keep it safe." Tom was happy for Sue and they walked home together, excited to read the special book.',
  'Once upon a time, there was a little dog named Spot. Spot loved to run and play in the park. One day, Spot saw a big, ugly cat. The cat had a treat in its mouth. Spot wanted the treat, so he ran after the cat.\n\n"Wait for me!" Spot barked, trying to hurry after the cat. But the cat was too fast and soon disappeared behind a tr

### IRREGULAR_PLURAL_SUBJECT_VERB_AGREEMENT_1

In [59]:
task = "irregular_plural_subject_verb_agreement_1"
baseline_scores = load_indexed_tensor(task)
top_variance_blimp(5, baseline_scores, task)

Creating new tensor


/tmp/ipykernel_3065793/965520794.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sorted_indices = torch.load(path+"sorted_indices.pt")
/tmp/ipykernel_3065793/965520794.

[{'sentence_good': "Some octopi weren't shocking Galileo.",
  'sentence_bad': "Some octopi isn't shocking Galileo.",
  'id': 499},
 {'sentence_good': "That analysis isn't irritating Winston Churchill.",
  'sentence_bad': "That analysis aren't irritating Winston Churchill.",
  'id': 325},
 {'sentence_good': 'Some radius aggravates Brad.',
  'sentence_bad': 'Some radius aggravate Brad.',
  'id': 114},
 {'sentence_good': "Some synthesis isn't aggravating Robert.",
  'sentence_bad': "Some synthesis weren't aggravating Robert.",
  'id': 74},
 {'sentence_good': 'Oxen irritate Monet.',
  'sentence_bad': 'Oxen irritates Monet.',
  'id': 341}]

In [60]:
top, bottom = examine_variance(n_blimp=5, n_stories=10, baseline_scores=baseline_scores, task=task)
print("TOP INFLUENTIAL STORIES\n")
top

TOP INFLUENTIAL STORIES



[["Nicky was walking outside and saw a puppy playing in the park. Nicky saw how friendly the puppy was and wanted to be friends. Nicky waved at the puppy and the puppy came running over to him.\n\nNicky: Hi puppy!\n\nPuppy: (Bark, bark!)\n\nNicky laughed and was so happy to make a new friend.\n\nNicky: Do you want to play?\n\nPuppy: (Bark, bark!)\n\nNicky and the puppy played together for a long time. Nicky and the puppy ran around and chased each other until they were both out of breath. Nicky was so happy and his friends were envious of him.\n\nNicky: I've gained a new best friend!\n\nPuppy: (Bark, bark!)",
  "Once upon a time there was a pig named Porky. Porky was happy and content, but one day he noticed something strange.\n\nPorky saw that all the other animals were eating the same food--grass, berries and fruit. But Porky noticed the humans ate something different--meat! Porky wondered why he was not allowed to eat it.\n\nSo Porky asked the farmer why he was not allowed to have t

In [61]:
print("BOTTOM INFLUENTIAL STORIES\n")
bottom

BOTTOM INFLUENTIAL STORIES



[['Once upon a time, there was a deer named Bambi. Bambi had a dull coat and was very shy. One day, Bambi saw a bird zoom past him in the sky. Bambi was amazed and wished he could zoom like the bird. \n\nBambi decided to practice running faster and faster. One day, while he was running, he saw a butterfly zoom past him. Bambi was so happy that he was able to zoom too, but in his own way. From that day on, Bambi felt more confident and less dull. He knew he could do anything if he tried hard enough.',
  'Once there was a little girl called Letsty. When Letsty was very confused, she always looked for her safe. But one day when Letsty couldn\'t find her safe, she asked her mum, "Where\'s my safe?". \n\nMum said, "It\'s okay, Letsty. I had to remove your safe to clean the house." \n\nLetsty was wondering what it meant to remove, so she asked her mum again. Her mum replied, "Remove means to take away something very carefully." \n\nLetsty understood what to do when she was confused. And she 

## TEMPLATE

In [15]:
task = "sentential_negation_npi_licensor_present"
# task = "npi_present_2"
baseline_scores = load_indexed_tensor(task)
top_variance_blimp(5, baseline_scores, task)


Loading existing tensor


/tmp/ipykernel_3085633/965520794.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  baseline_scores = torch.load(path+"baseline_scores.pt")


[{'sentence_good': 'Florence has not ever hugged Debra.',
  'sentence_bad': 'Florence has fortunately ever hugged Debra.',
  'id': 432},
 {'sentence_good': 'Raymond should not ever fail to aggravate Benjamin.',
  'sentence_bad': 'Raymond should really ever fail to aggravate Benjamin.',
  'id': 81},
 {'sentence_good': 'Winston Churchill can not ever expect every shirt to crumple.',
  'sentence_bad': 'Winston Churchill can probably ever expect every shirt to crumple.',
  'id': 325},
 {'sentence_good': 'Gregory might not ever obligate the Lutherans to confer.',
  'sentence_bad': 'Gregory might really ever obligate the Lutherans to confer.',
  'id': 118},
 {'sentence_good': 'Tiffany can not ever persuade Wayne to eat.',
  'sentence_bad': 'Tiffany can fortunately ever persuade Wayne to eat.',
  'id': 485}]

In [16]:
top, bottom = examine_variance(n_blimp=5, n_stories=10, baseline_scores=baseline_scores, task=task)
print("TOP INFLUENTIAL STORIES\n")
top

TOP INFLUENTIAL STORIES



[["Once upon a time, there was a happy spirit named Sparky. Sparky loved to play in the light and turn around in circles. One day, Sparky saw a sad spirit who was feeling down. Sparky wanted to help cheer up the sad spirit, so Sparky started to dance around and turn around in circles. The sad spirit started to smile and feel happy again! Sparky's light and happy spirit had helped the sad spirit feel better. From that day on, the two spirits became friends and would often turn around in circles together in the light.",
  "Once upon a time, there was a little rabbit named Hoppy. Hoppy loved to perform on stage. He was very clever and could do many tricks. One day, Hoppy saw a little bird who was sad because she couldn't fly. Hoppy felt bad for her and decided to help.\n\nHoppy taught the little bird how to hop and jump on the stage. The little bird was so happy and grateful. She performed with Hoppy and they became great friends. \n\nHoppy learned that it's not just about performing on s

In [17]:
print("BOTTOM INFLUENTIAL STORIES\n")
bottom

BOTTOM INFLUENTIAL STORIES



[['Once there was a little girl called Letsty. When Letsty was very confused, she always looked for her safe. But one day when Letsty couldn\'t find her safe, she asked her mum, "Where\'s my safe?". \n\nMum said, "It\'s okay, Letsty. I had to remove your safe to clean the house." \n\nLetsty was wondering what it meant to remove, so she asked her mum again. Her mum replied, "Remove means to take away something very carefully." \n\nLetsty understood what to do when she was confused. And she felt safe knowing that her mum would always look after her and remove her worries.\n\nThe moral of the story is that no matter how confused we are, we can always rely on our family to help us through it.',
  "Once upon a time, there was a boy named Tim. Tim had a big, heavy box. He hoped the box had money in it. He wanted to buy a toy with the money.\n\nOne day, Tim's friend, Sam, saw the heavy box. Sam wanted the money too. They both wanted the toy. They did not know how to share the toy. This made t